# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [143]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [144]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [145]:
class process:
    def __init__(self, file, encoding):
        self.file = file
        self.encoding = encoding

    def read(self, size):
        return self.file.read(size).encode(self.encoding)

    def readline(self, size=-1):
        return self.file.readline(size).encode(self.encoding)

    def write(self, binaryData):
        return self.file.write(FileWrapper(binaryData[1:].decode(binaryData, self.encoding)))

    
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(process(open(filename), 'utf-8'))  
    # data = pickle.load(open(filename, 'rb'))     
    target = [i] * len(data)
    return data, target

In [146]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
print ('total number of data', len(all_data))

total number of data 222


In [147]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

In [148]:
print(training_data)

[113  56 180  57  76 145 151  32  43 213  12 189   9 159 107 185 218 179
  37 136 139  93  18 216  92 149  68  27  15 105  52 137 204 220   2 175
 118 128   7   0 167  96  70 197  30  63  66  88 142  40  81  24  72 133
 100 112 209   4 219 102  54 124  44  26 103 214  64  17 132  31  49   1
  61   3 187  87  38 157 122  71 121  20 119 125  67  80 123 143 163  42
 165 198  99 111   6  14 191  58 138 108 221 114 147  48  60 181 158  84
 206 190  41  28  10 135 201 115  25  91 169  89 196 199 154  53  13 150
 170  74 171 192  47  86 212 178 155  21  45 211 153 160  83  78 207 193
  19 152   8 217  22 106  39  73 168 161  65]


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [149]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [166]:
train_data, test_data, train_target, test_target = train_test_split(all_data, all_target, test_size=0.3)

clf.fit(train_X, train_Y)


AttributeError: 'function' object has no attribute 'fit'

### predicting

In [151]:
clf.predict([train_X[-1]]), train_Y[-1]

(array([6]), 6)

In [152]:
# clf.predict(all_data[-1]), all_target[-1]

In [153]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [154]:
expected = []
predicted = []
# YOUR CODE HERE
for i in range(len(all_data)):
    expected.append(all_target[i])

predicted = clf.predict(all_data)
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        19
           2       0.97      1.00      0.98        30
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        20
           7       1.00      0.91      0.95        11
           8       1.00      1.00      1.00        26
           9       1.00      1.00      1.00        11
          10       1.00      1.00      1.00        52

    accuracy                           1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[23  0  0  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0  0  0]
 [ 0  0 30  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [155]:
expected = []
predicted = []
# YOUR CODE HERE
expected=test_Y
predicted=clf.predict(test_X)
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00        10
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00        13

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 6  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [156]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [157]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[:-1]), np.array(all_target[:-1])

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  2,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]),
 array([ 0